# Atencja — implementacja atencji między danym stanem dekodera a stanami kodera

Poniżej znajdują się dwie macierze `encoder_states` i `decoder_states` reprezentujące stan warstwy ukrytej po przetworzeniu każdego słowa przez koder oraz statyczne wektory zanurzeń związane z danym wejściem dekodera. Pojedynczy stan warstwy ukrytej zawiera zanurzenia o długości = 3, która to liczba jest równa rozmiarowi zanurzenia w dekoderze. W koderze mamy 4 stany warstw ukrytych, ponieważ przetwarzamy sekwencję składającą się z 4 tokenów.

W dekoderze znajduje się 5 tokenów, które są generowane na podstawie sekwencji przetwarzanej przez koder.

# Zadanie 1 (1 punkt)

Zadanie polega na: a) Obliczeniu podobieństwa wszystkich zanurzeń z dekodera (zapytań -- queries) do wszystkich zanurzeń kolejnych stanów kodera (kluczy -- keys) (pamiętaj, że macierze można transponować. W NumPy transponujemy macierz za pomocą `nazwa_macierzy. T`)

a) Softmax (zaimportowany z scipy) należy wykonać na utworzonej macierzy podobieństw. Uwaga: pamiętaj, aby zastosować softmax we właściwym wymiarze. Wszystkie ukryte stany kodera powinny być zmiękczone w kontekście danego stanu dekodera. W scipy funkcja softmax zawiera argument `axis`, który może pomóc.

b) Połącz macierz uwagi z kroku b) i „encoder_states”, aby wygenerować macierz zawierającą wektory kontekstu dla każdego tokena z dekodera.

In [ ]:
import numpy as np
from scipy.special import softmax

# scipy.special.softmax(x, axis=None)

encoder_states = np.array(
    [[1.2, 3.4, 5.6],   # encoder's hidden layer output at the step 1, related to a given input token, e.g., I
    [-2.3, 0.2, 7.2],   # encoder's hidden layer output at the step 2, related to a given token, e.g., like
    [10.2, 0.2, 0.3],   # encoder's hidden layer output at the step 3, related to a given token, e.g., NLP
    [0.4, 0.7, 1.2]]    # encoder's hidden layer output at the step 4, related to a given token, e.g., "."
)

decoder_states = np.array(
    [[0.74, 0.23, 0.56],  # decoder's static word embedding at the step 1, related to a given token, e.g., <BOS>
    [7.23, 0.12, 0.55],  # decoder's static word embedding at the step 2, related to a given token, e.g., Ja
    [9.12, 4.23, 0.44], # decoder's static word embedding at the step 3, related to a given token, e.g., lubię
    [4.1, 3.23, 0.5],    # decoder's static word embedding at the step 4, related to a given token, e.g., przetwarzanie
    [5.2, 3.1, 8.5]]     # decoder's static word embedding at the step 5, related to a given token, e.g., języka
)

# Similarity = dot product
attention_scores = np.matmul(decoder_states, encoder_states.T)
print(attention_scores)

attention_weights = softmax(attention_scores, axis=1)
print(attention_weights)

context_vectors = np.matmul(attention_weights, encoder_states)
print(context_vectors)

[[  4.806   2.376   7.762   1.129]
 [ 12.164 -12.645  73.935   3.636]
 [ 27.79  -16.962  94.002   7.137]
 [ 18.702  -5.184  42.616   4.501]
 [ 64.38   49.86   56.21   14.45 ]]
[[4.91780633e-02 4.32948093e-03 9.45248312e-01 1.24414389e-03]
 [1.49003187e-27 2.50486173e-38 1.00000000e+00 2.94803216e-31]
 [1.75587568e-29 6.44090821e-49 1.00000000e+00 1.88369172e-38]
 [4.11416552e-11 1.74069934e-21 1.00000000e+00 2.79811669e-17]
 [9.99716568e-01 4.94220792e-07 2.82937800e-04 2.06801368e-22]]
[[ 9.69108631  0.35799187  0.59163688]
 [10.2         0.2         0.3       ]
 [10.2         0.2         0.3       ]
 [10.2         0.2         0.3       ]
 [ 1.20254471  3.39909302  5.59850122]]


Oczekiwane wyjścia:

a) [[ 4.806 2.376 7.762 1.129] [ 12.164 -12.645 73.935 3.636] [ 27.79 -16.962 94.002 7.137] [ 18.702 -5.184 42.616 4.501] [ 64.38 49.86 56.21 14.45 ]]

b) [[4.91780633e-02 4.32948093e-03 9.45248312e-01 1.24414389e-03] [1.49003187e-27 2.50486173e-38 1.00000000e+00 2.94803216e-31] [1.75587568e-29 6.44090821e-49 1.00000000e+00 1.88369172e-38] [4.11416552e-11 1.74069934e-21 1.00000000e+00 2.79811669e-17] [9.99716568e-01 4.94220792e-07 2.82937800e-04 2.06801368e-22]]

c) [[ 9.69108631 0.35799187 0.59163688] [10.2 0.2 0.3 ] [10.2 0.2 0.3 ] [10.2 0.2 0.3 ] [ 1.20254471 3.39909302 5.59850122]]


# Transformer
## Wykorzystanie modelu T5 opartego na transformerze do rozwiązywania różnych zadań NLP.

Dzisiaj poznamy nową bibliotekę — HuggingFace **transformers** (https://huggingface.co/docs/transformers/index) i użyjemy jej do rozwiązania kilku nieoczywistych problemów związanych z NLP za pomocą modelu **T5** .


HuggingFace transformers to jedna z najpopularniejszych bibliotek dostarczających nam wysokopoziomowe API do wykorzystania sieci neuronowych do rozwiązywania zadań związanych z przetwarzaniem języka naturalnego, przetwarzaniem dźwięku, wizją komputerową, a nawet scenariuszami multimodalnymi, w których musimy wykorzystywać wiele modalności naraz (np. odpowiadając na pytania o zdjęcia, wydobywając informacje z faktur).

Najpierw zainstalujmy zależności, samą bibliotekę `transformers` oraz moduł `sentencepiece`, który pomaga nam tokenizować dokumenty i przekształcać tokeny w kodowanie one-hot (ideę sentencepiece omówimy szczegółowo później).

**Ostrzeżenie**: jeśli zauważysz jakieś dziwne wyjątki, takie jak `cannot call from_pretrained na obiekcie None` gdzieś w twoim kodzie, zrestartuj środowisko używając: Runtime -> restart. Następnie uruchom komórki z kodem (bez ponownej instalacji bibliotek) jeszcze raz.

In [ ]:
!pip install transformers   # install HuggingFace transformers library
!pip install sentencepiece  # install sentencepiece

Przeczytaj dokumentację dotyczącą modelu T5 dostępne tutaj: https://huggingface.co/docs/transformers/model_doc/t5

W sekcji `wnioskowanie` znajdziesz opis pokazujący w jaki sposób możemy pobrać wytrenowany model i użyć go do rozwiązania zadanego zadania. Po prostu użyj dostarczonego kodu, aby przetłumaczyć jakieś zdanie z angielskiego na niemiecki!

*(0.5 punkta)*

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

input_ids = tokenizer("translate English to German: Just leave me alone, I know what I'm doing.", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Lassen Sie mich einfach alleine, ich weiß, was ich tun.


## Różne zadania

Eksperymentuj z innymi danymi wejściowymi, np. tymi, które przedstawiono na rysunku 1 przedstawionym w artykule przedstawiającym model T5 lub nawet szerszą listą przypadków użycia z Dodatku D dostarczonego z artykułem. Artykuł można znaleźć tutaj: https://arxiv.org/pdf/1910.10683.pdf

Uwaga: wśród dostarczonych danych wejściowych zastosowano kilka skrótów, niektóre z nich to:
- `stsb`: oznacza semantyczny test porównawczy podobieństwa tekstu. Biorąc pod uwagę dwa zdania, możemy obliczyć ich podobieństwa semantyczne, co może pomóc nam ustalić, czy jedno zdanie jest parafrazą drugiego.
- `cola`: oznacza Corpus of Linguistic Acceptability i pomaga nam określić, czy dane zdanie jest gramatyczne, czy niegramatyczne.

Jeśli spojrzysz na Dodatek D, skrótów jest więcej, są one związane z nazwami zadań przedstawionymi w benchmarku GLUE (dostępny tutaj: https://gluebenchmark.com/tasks) i benchmarku SUPERGLUE (dostępny tutaj: https:/ /super.gluebenchmark.com/tasks). Ideą GLUE i SUPERGLUE jest zebranie zestawu trudnych zadań, które można wykorzystać do oceny systemów wymagających rozumienia języka naturalnego.

**Wklej 3 przykładowe zadania i przetworzone dane wejściowe w komórce poniżej (1 punkt)**

In [ ]:
input_1 = "stsb sentence1: The cat is sitting on the mat. sentence2: The cat is resting on the rug."
input_2 = "cola sentence: He didn't went to the store."
input_3 = "paraphrase sentence1: They're having a party! sentence2: A party is happening."

# Tokenize input data
inputs_1 = tokenizer(input_1, return_tensors="pt", padding=True, truncation=True)
inputs_2 = tokenizer(input_2, return_tensors="pt", padding=True, truncation=True)
inputs_3 = tokenizer(input_3, return_tensors="pt", padding=True, truncation=True)

outputs_1 = model.generate(inputs_1.input_ids)
outputs_2 = model.generate(inputs_2.input_ids)
outputs_3 = model.generate(inputs_3.input_ids)

decoded_output_1 = tokenizer.decode(outputs_1[0], skip_special_tokens=True)
decoded_output_2 = tokenizer.decode(outputs_2[0], skip_special_tokens=True)
decoded_output_3 = tokenizer.decode(outputs_3[0], skip_special_tokens=True)

print("STSB result:", decoded_output_1)
print("Grammar check result:", decoded_output_2)
print("Paraphrase result:", decoded_output_3)

STSB result: 2.4
Grammar check result: acceptable
Paraphrase result: True


## Różne typy modeli

Dostępnych jest kilka modeli T5, które różnią się wielkością (i jakością). Im większy model, tym lepsze wyjście powinien generować. Eksperymentuj z niektórymi modelami z następującego zestawu:
- t5-small
- base t5
- t5-large
- t5-3b
- t5-11b

Sprawdź, czy można zaobserwować różnicę w jakości generowanych tekstów.

Porównaj również rozmiar modeli, możesz użyć funkcji `model.num_parameters()`, aby uzyskać numer parametru związany z każdym modelem. Dla każdego modelu, który jesteś w stanie załadować, podaj rozmiar w poniższej komórce (jeśli nie możesz załadować danego modelu, bo jest za duży, bez obaw, po prostu wpisz 'too big to load'). (*2 punkty*)

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Lista modeli T5 do eksperymentowania
models = ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]

for model_name in models:
    try:
        # Ładowanie modelu i tokenizatora
        tokenizer = T5Tokenizer.from_pretrained(model_name)
        model = T5ForConditionalGeneration.from_pretrained(model_name)

        # Liczenie parametrów modelu
        num_params = model.num_parameters()

        print(f"{model_name} params number: {num_params}")
    except:
        print(f"{model_name} - too big to load")


# t5-small params number: 60506624
# t5-base params number: 222903552
# t5-large params number: 737668096
# t5-3b params number: too big to load: 2.85B (source: https://huggingface.co/google-t5/t5-3b)
# t5-11b params number: too big to load: 11B (source: https://huggingface.co/google-t5/t5-11b)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/11.4G [00:00<?, ?B/s]

## T5 specyficzne dla języka (ZADANIE OPCJONALNE – nie musisz tutaj podawać kodu)

Istnieją nawet alternatywy dla oryginalnych modeli T5. Ponieważ model T5 był trenowany na języku angielskim, dostępne są modele specyficzne dla innych języków, np. polskiego (np. plT5 zaproponowany przez Allegro - https://huggingface.co/allegro/plt5-small). Polski model został przeszkolony do rozwiązywania zestawu zadań zebranych w benchmarku KLEJ, który stanowi polską analogię do benchmarku GLUE: https://klejbenchmark.com.

Więcej szczegółów na temat plT5 można znaleźć w artykule badawczym: https://arxiv.org/pdf/2205.08808.pdf. Tabela 2 przedstawia przykładowe podpowiedzi, które można wykorzystać do rozwiązania niektórych zadań wymienionych w KLEJ.

Możesz wyszukać alternatywę dla oryginalnego T5, na przykład tę związaną z Twoim językiem, i poeksperymentować z nią (**to zadanie nie jest obowiązkowe**).

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Wczytanie tokenizer'a i modelu plT5
tokenizer = T5Tokenizer.from_pretrained("allegro/plt5-small")
model = T5ForConditionalGeneration.from_pretrained("allegro/plt5-small")

# Zadania do wykonania
tasks = [
    "Tłumaczenie z angielskiego na polski: Hello, how are you?",
    "Podsumowanie artykułu: Albert Einstein był niemiecko-urodzonym fizykiem teoretycznym, który opracował teorię względności, jedną z dwóch filarów współczesnej fizyki...",
    "Generowanie nagłówka dla następującego artykułu: Naukowcy odkrywają nowy gatunek motyla w deszczowym lesie Amazonii...",
    "Rozwiązanie problemu matematycznego: Jakie jest pierwiastek kwadratowy z 144?",
    "Opisanie fabuły filmu Titanic.",
]

# Wykonanie zadań
for task in tasks:
    # Tokenizacja zadania
    inputs = tokenizer(task, return_tensors="pt", padding=True, truncation=True)
    # Generowanie outputu
    outputs = model.generate(**inputs)
    # Dekodowanie outputu
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Wyświetlenie wyniku
    print("Zadanie:", task)
    print("Wynik:", output_text)
    print()

# I tried, but I'm still not getting any Polish characters


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Zadanie: Tłumaczenie z angielskiego na polski: Hello, how are you?
Wynik: 規.

Zadanie: Podsumowanie artykułu: Albert Einstein był niemiecko-urodzonym fizykiem teoretycznym, który opracował teorię względności, jedną z dwóch filarów współczesnej fizyki...
Wynik: 規.̋..誰.......................

Zadanie: Generowanie nagłówka dla następującego artykułu: Naukowcy odkrywają nowy gatunek motyla w deszczowym lesie Amazonii...
Wynik: 規.̋................................

Zadanie: Rozwiązanie problemu matematycznego: Jakie jest pierwiastek kwadratowy z 144?
Wynik: 規.̋, a nie matematyka.

Zadanie: Opisanie fabuły filmu Titanic.
Wynik: 規



## Flan-T5

Pod koniec 2022 roku zaproponowano ewolucję T5 o nazwie Flan-T5. Ten model jest również dostarczany przez bibliotekę transformatorów HuggingFace. Odwiedź tę stronę: https://huggingface.co/docs/transformers/model_doc/flan-t5, aby zobaczyć, jak możesz użyć tego modelu (wystarczy zmienić nazwę modelu, aby pobrać!).

Flan-T5 jest znacznie potężniejszy niż T5. Możesz zajrzeć do Dodatku D zawartego w artykule opisującym Flan T5, aby zapoznać się z niektórymi formatami wejściowymi (monitami) i generowanymi wartościami. Artykuł jest tutaj: https://arxiv.org/pdf/1910.10683.pdf. Powinieneś skupić się na polach „przetworzonych danych wejściowych”, ponieważ są to reprezentacje używane przez model. Eksperymentuj z wybranymi zadaniami i sprawdź, czy możesz uzyskać takie same wyniki! W poniższym kodzie wklej kod ładujący model Flan-T5 i wykorzystujący go do rozwiązania wybranych zadań. (*1 punkt*)

In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Wczytanie modelu Flan-T5 i tokenizatora
model_name = "google/flan-t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Zadania do wykonania
tasks = [
    "Summarize the following article: Albert Einstein was a German-born theoretical physicist who developed the theory of relativity, one of the two pillars of modern physics...",
    "Transltion from English to French of the phrase Hello world:",
    "Headline for the following news article: Scientists discover new species of butterfly in the Amazon rainforest...",
    "The square root of 144?",
    "The plot of the movie Titanic.",
    "A step by step recipe to make bolognese pasta:",
]

# Wykonanie zadań
for task in tasks:
    # Tokenizacja zdania
    inputs = tokenizer(task, return_tensors="pt")
    # Generowanie outputu
    outputs = model.generate(**inputs)
    # Dekodowanie outputu
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Wyświetlenie wyniku
    print("Task:", task)
    print("Output:", output_text)
    print()


Task: Summarize the following article: Albert Einstein was a German-born theoretical physicist who developed the theory of relativity, one of the two pillars of modern physics...
Output: Einstein was born in Munich, Germany, and studied at the University of Heidelberg. He studied

Task: Transltion from English to French of the phrase Hello world:
Output: Transltion de l'anglais à l'anglais

Task: Headline for the following news article: Scientists discover new species of butterfly in the Amazon rainforest...
Output: The sandstone sandstone sandstone is a rare species of

Task: The square root of 144?
Output: 144

Task: The plot of the movie Titanic.
Output: The plot of Titanic is based on the story of a young woman who is 

Task: A step by step recipe to make bolognese pasta:
Output: Pour a cup of bolognese into a large bowl and add the pasta



## (OPCJONALNIE) Dostrajanie

Możesz nawet dostroić model T5/Flan-T5, aby rozwiązać wybrane zadanie. Możesz załadować istniejący model T5/Flan-T5, który jest już przeszkolony do rozwiązywania niektórych zadań, i wykorzystać moc 'transfery wiedzy', aby nauczyć go rozwiązywać różne zadania. Jest to o wiele lepsze niż trenowanie sieci od zera i powinno wymagać mniejszej liczby przykładów szkoleniowych.

Faza dostrajania jest dość złożona. Jednak opis krok po kroku można znaleźć tutaj: https://www.philschmid.de/fine-tune-flan-t5

Możesz spróbować dostroić wybrany model.

In [ ]:
!pip install datasets
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch

# Load the pre-trained T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Load the dataset
dataset = load_dataset("wikipedia", "20220301.en", split="train[:100]")  # Load first 100 examples for demonstration

# Preprocess the dataset
train_texts = dataset["text"]
train_labels = dataset["title"]

# Tokenize training data and labels
train_encodings = tokenizer(train_texts, padding=True, truncation=True, return_tensors="pt")
train_labels_encodings = tokenizer(train_labels, padding=True, truncation=True, return_tensors="pt")

training_args = TrainingArguments(
    output_dir="./output_dir",  # Directory for storing outputs
    per_device_train_batch_size=2,    # Batch size per GPU
    num_train_epochs=3,               # Number of training epochs
    logging_dir="./logs",             # Directory for storing logs
)

# Define Trainer
trainer = Trainer(
    model=model,                      # The instantiated Transformers model to be trained
    args=training_args,               # Training arguments
    train_dataset=train_encodings,   # Training dataset
    eval_dataset=train_labels_encodings,    # Evaluation dataset
)

# Train the model
trainer.train()

# Generate predictions using the fine-tuned model
input_text = "translate English to German: Hello, how are you?"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs)
generated_translation = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the generated translation
print("Generated translation:", generated_translation)

